![lyon2 geonum](https://perso.liris.cnrs.fr/lmoncla/GEONUM/fig/logos.png)

# 2F2 – Gestion et traitement des données spatio-temporelles1


## Tutoriel : Analyse des données des disponibilités des stations Vélo'v de la Métropole de Lyon


# Partie 2 : Analyse et visualisation des données


Dans le cadre de ce TP,  vous avez à votre disposition l'ensemble des données pour l'année 2021.


Les objectifs de cette partie sont les suivants : 

* Analyser les données : manipuler les opérations de regroupement de la librairies Pandas.




# Module GroupBy - Pandas (30 minutes)

**Colonnes disponibles :**
- `id_velov` : identifiant de la station
- `year`, `month`, `day`, `hour`, `minute` : informations temporelles
- `bikes` : nombre de vélos disponibles
- `bike_stands` : nombre d'emplacements disponibles
- `departure30min` : nombre de départs sur 30 minutes
- `arrival30min` : nombre d'arrivées sur 30 minutes
- `daily_departure`, `daily_arrival` : totaux journaliers
- `IsWeekday` : True si jour de semaine
- `day_of_week` : numéro du jour (0=lundi, 6=dimanche)

## 1. Installation et importation des libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timezone
import wget
import folium
import plotly
import plotly.express as px
import geopandas

## 2. Récupération du jeu de données

Dans un premier temps il faut récupérer les données. D'un côté le jeu de données contenant les localisations des stations et de l'autre l'historique d'utilisation. Le second a été modifié lors de la précédente séance. Pour ne pas avoir à refaire tous les traitements vous pouvez récupérer directement l'archive `data-bikes-2.zip`.

L'ensemble des données utilisées dans ce tutoriel est disponible à cette adresse : 
https://perso.liris.cnrs.fr/lmoncla/GEONUM/

* Télécharger les archives contenant les données
1. data-stations.zip
2. data-bikes-2.zip

Ces 2 archives contiennent chacune un fichier CSV contenant respectivement la liste des stations vélov (et leur localisation) et la liste des disponibilités de chaque station par tranche de 30 minutes.


In [ ]:
## On télécharge l'archive contenant la liste des stations
wget.download("https://perso.liris.cnrs.fr/lmoncla/GEONUM/data-stations.zip",out="../data/")
    
## On télécharge l'archive contenant la liste des disponibilité des stations par tranche de 5 minutes
wget.download("https://perso.liris.cnrs.fr/lmoncla/GEONUM/data-bikes-2.zip",out="../data")

### 2.1. Chargement des données

Comme la dernière fois, pour charger les données il suffit d'utiliser la méthode [read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html#pandas.read_csv) de la librairie `Pandas`. Elle prend en paramètre le chemin du fichier que l'on souhaite charger. Ce fichier peut être de 2 formats, soit directement un fichier CSV, soit un fichier ZIP contenant un CSV. Dans notre, cas il est donc inutile de dézipper les archives téléchargées précédemment.


In [ ]:
## On charge les données des stations dans un dataframe
df_stations = pd.read_csv('../data/data-stations.zip')

## On crée maintenant le dataframe avec les données d'historique
df_bikes = pd.read_csv('../data/data-bikes-2.zip')

In [ ]:
## On affiche les premières lignes
df_bikes.head()

### 2.2. Premier apercu des données d'historique

In [ ]:
## On affiche les information sur les données
df_bikes.info()

In [ ]:
# Réduction de la taille en mémoire

## on transforme le type des colonnes en entier ou float lorsque cela est nécessaire
#df_bikes.bikes = df_bikes.bikes.apply(lambda x: int(float(x)))
#df_bikes.bike_stands = df_bikes.bike_stands.apply(lambda x: np.int32(float(x)))

df_bikes[['year', 'daily_departure', 'daily_arrival']] = df_bikes[['year', 'daily_departure', 'daily_arrival']].astype('int16')
df_bikes[['month','day','hour','minute', 'bikes', 'bike_stands', 'departure30min','arrival30min']] = df_bikes[['month','day','hour','minute', 'bikes', 'bike_stands', 'departure30min','arrival30min']].astype('int8')
## On affiche les information sur les données
df_bikes.info()

---

## 3. Concept de base du GroupBy

### Le principe en 3 etapes : Diviser - Appliquer - Combiner

1. **Diviser** : divise les données en groupes selon une/des colonne(s)
2. **Appliquer** : applique une fonction d'agrégation à chaque groupe
3. **Combiner** : combine les résultats en un seul objet

```python
df.groupby('colonne').fonction_agregation()
```

### Exemple simple

In [ ]:
# Nombre moyen de vélos disponibles par heure de la journée
df_bikes.groupby('hour')['bikes'].mean()

In [ ]:
# Nombre total de départs par jour de la semaine
df_bikes.groupby('day_of_week')['departure30min'].sum()

---

## 2. Agrégations simples

### Fonctions d'agrégation courantes

- `mean()` : moyenne
- `sum()` : somme
- `count()` : nombre d'éléments
- `min()`, `max()` : minimum, maximum
- `std()` : écart-type
- `median()` : médiane

In [ ]:
# Plusieurs statistiques en même temps avec .agg()
df_bikes.groupby('hour')['bikes'].agg(['mean', 'min', 'max', 'std'])

In [ ]:
# Compter le nombre d'enregistrements par mois
df_bikes.groupby('month')['id_velov'].count()

### EXERCICE 1 

**Objectif :** Trouvez le nombre moyen de `bike_stands` disponibles pour chaque mois.


In [ ]:
# LE CODE ICI


---

## 3. Groupements multiples

### Grouper selon plusieurs colonnes

On peut grouper selon plusieurs critères en passant une liste de colonnes.

In [ ]:
# Moyenne de vélos par type de jour (semaine/week-end) ET par heure
df_bikes.groupby(['IsWeekday', 'hour'])['bikes'].mean()

In [ ]:
# Agrégation sur plusieurs colonnes en même temps
df_bikes.groupby(['year', 'month'])[['departure30min', 'arrival30min']].mean()

### Réinitialiser l'index avec `.reset_index()`

Par défaut, les colonnes de groupement deviennent l'index. Pour obtenir un DataFrame "classique" :

In [ ]:
# Avec index multi-niveau (par défaut)
result_avec_index = df_bikes.groupby(['year', 'month'])['daily_departure'].sum()
print("Type:", type(result_avec_index))
result_avec_index.head()

In [ ]:
# Avec reset_index() - plus facile à manipuler
result_dataframe = df_bikes.groupby(['year', 'month'])['daily_departure'].sum().reset_index()
print("Type:", type(result_dataframe))
result_dataframe.head()

### EXERCICE 2

**Objectif :** Calculez le nombre total de départs (`departure30min`) par mois ET par jour de la semaine. N'oubliez pas de réinitialiser l'index !

In [ ]:
# CODE ICI


---

## 4. Agrégations avancées

### Utiliser `.agg()` avec un dictionnaire

Pour appliquer des fonctions différentes à différentes colonnes :

In [ ]:
# Plusieurs fonctions sur plusieurs colonnes
df_bikes.groupby('hour').agg({
    'bikes': ['mean', 'max'],
    'bike_stands': ['mean', 'min'],
    'departure30min': 'sum'
})

### Fonctions personnalisées avec lambda

In [ ]:
# Calculer l'amplitude (max - min) de vélos disponibles par heure
df_bikes.groupby('hour')['bikes'].agg(lambda x: x.max() - x.min())

### Cas d'usage : Identifier les heures de pointe

In [ ]:
# Top 5 des heures avec le plus de départs
heures_pointe = df_bikes.groupby('hour')['departure30min'].sum().sort_values(ascending=False).head(5)
print("Heures de pointe (départs) :")
print(heures_pointe)

### Comparaison semaine / week-end avec `.unstack()`

In [ ]:
# Départs moyens par heure selon le type de jour
comparison = df_bikes.groupby(['IsWeekday', 'hour'])['departure30min'].mean().unstack()
# .unstack() transforme en tableau : IsWeekday en lignes, hour en colonnes
comparison

### EXERCICE 3

**Objectif :** Trouvez les 3 stations (`id_velov`) qui ont le plus grand nombre total d'arrivées (`arrival30min`) sur toute la période.

In [ ]:
# CODE ICI


---

## 5. Cas pratique : Profil d'usage des stations

### Analyse complète de l'utilisation

In [ ]:
# 1. Profil d'activité par jour de semaine et heure
profile = df_bikes.groupby(['day_of_week', 'hour']).agg({
    'departure30min': 'sum',
    'arrival30min': 'sum',
    'bikes': 'mean'
}).reset_index()

profile.head(10)

In [ ]:
# 2. Trouver l'heure de pointe de chaque jour de la semaine
idx_max = profile.groupby('day_of_week')['departure30min'].idxmax() ## retourne l'identifiant de la ligne avec la valeur MAX
heures_pointe_par_jour = profile.loc[idx_max,:]
print("Heure de pointe par jour de semaine :")
print(heures_pointe_par_jour)
